In [1]:
import sys
sys.path.insert(0, '../')
import numpy as np
%reload_ext autoreload
%autoreload 1
%aimport allel
%aimport allel.model.ndarray.arrays
%aimport allel.model.ndarray.tables
%aimport allel.model.ndarray.indexes
allel.__version__

'0.22.0.dev0'

## GenotypeVector

In [2]:
g = allel.GenotypeVector([[0, 1], [-1, -1]]*20)
g.is_phased = np.array([True, False]*20, dtype=bool)
g

0,1,2,3,4,...,35,36,37,38,39
0|1,./.,0|1,./.,0|1,...,./.,0|1,./.,0|1,./.


In [3]:
g.display(30, 10)

0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,...,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.


In [4]:
g.displayall()

0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39
0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.


In [5]:
print(repr(g))

GenotypeVector((40, 2), dtype=int64)
0|1 ./. 0|1 ./. 0|1  ...  ./. 0|1 ./. 0|1 ./.


In [6]:
print(g)

0|1 ./. 0|1 ./. 0|1  ...  ./. 0|1 ./. 0|1 ./.


In [7]:
print(g.to_str(40))

0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./.


In [8]:
g.vstack(g)

0,1,2,3,4,...,75,76,77,78,79
0/1,./.,0/1,./.,0/1,...,./.,0/1,./.,0/1,./.


In [9]:
g.hstack(g)

array([[ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1]])

In [10]:
g.concatenate(g)

0,1,2,3,4,...,75,76,77,78,79
0/1,./.,0/1,./.,0/1,...,./.,0/1,./.,0/1,./.


## GenotypeArray

In [11]:
g = allel.GenotypeArray([[[0, 1], [-1, -1]]*20]*20)
g.is_phased = np.array([[True, False, False, False]*10]*20, dtype=bool)
g.mask = np.array([[1, 0, 0, 0, 0]*8]*20, dtype=bool)

In [12]:
print(g)

.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
...
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.



In [13]:
print(repr(g))

GenotypeArray((20, 40, 2), dtype=int64)
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
...
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.



In [14]:
g

GenotypeArray((20, 40, 2), dtype=int64)
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
...
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.

In [15]:
g.display(row_threshold=10, row_edgeitems=5, col_threshold=20, col_edgeitems=10)

GenotypeArray((20, 40, 2), dtype=int64) 0 1 2 3 4 5 6 7 8 9 ... 30 31 32 33 34 35 36 37 38 39 0 .|. ./. 0/1 ./. 0|1 ./. 0/1 ./. 0|1 ./. ... ./. ./. 0|1 ./. 0/1 ./. 0|1 ./. 0/1 ./. 1 .|. ./. 0/1 ./. 0|1 ./. 0/1 ./. 0|1 ./. ... ./. ./. 0|1 ./. 0/1 ./. 0|1 ./. 0/1 ./. 2 .|. ./. 0/1 ./. 0|1 ./. 0/1 ./. 0|1 ./. ... ./. ./. 0|1 ./. 0/1 ./. 0|1 ./. 0/1 ./. 3 .|. ./. 0/1 ./. 0|1 ./. 0/1 ./. 0|1 ./. ... ./. ./. 0|1 ./. 0/1 ./. 0|1 ./. 0/1 ./. 4 .|. ./. 0/1 ./. 0|1 ./. 0/1 ./. 0|1 ./. ... ./. ./. 0|1 ./. 0/1 ./. 0|1 ./. 0/1 ./. ... ... 15 .|. ./. 0/1 ./. 0|1 ./. 0/1 ./. 0|1 ./. ... ./. ./. 0|1 ./. 0/1 ./. 0|1 ./. 0/1 ./. 16 .|. ./. 0/1 ./. 0|1 ./. 0/1 ./. 0|1 ./. ... ./. ./. 0|1 ./. 0/1 ./. 0|1 ./. 0/1 ./. 17 .|. ./. 0/1 ./. 0|1 ./. 0/1 ./. 0|1 ./. ... ./. ./. 0|1 ./. 0/1 ./. 0|1 ./. 0/1 ./. 18 .|. ./. 0/1 ./. 0|1 ./. 0/1 ./. 0|1 ./. ... ./. ./. 0|1 ./. 0/1 ./. 0|1 ./. 0/1 ./. 19 .|. ./. 0/1 ./. 0|1 ./. 0/1 ./. 0|1 ./. ... ./. ./. 0|1 ./. 0/1 ./. 0|1 ./. 0/1 ./.

In [16]:
g.displayall()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39
0,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
1,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
2,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
3,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
4,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
5,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
6,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
7,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
8,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
9,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.


In [17]:
g[0]

0,1,2,3,4,...,35,36,37,38,39
.|.,./.,0/1,./.,0|1,...,./.,0|1,./.,0/1,./.


In [18]:
g[:, 4]

0,1,2,3,4,...,15,16,17,18,19
0|1,0|1,0|1,0|1,0|1,...,0|1,0|1,0|1,0|1,0|1


In [19]:
g[:, 4].displayall()

0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1


In [20]:
g.mask = None
g

GenotypeArray((20, 40, 2), dtype=int64)
0|1 ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
0|1 ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
0|1 ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
...
0|1 ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
0|1 ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.
0|1 ./. 0/1 ./. 0|1  ...  ./. 0|1 ./. 0/1 ./.

In [21]:
g.displayall()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39
0,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
1,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
2,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
3,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
4,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
5,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
6,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
7,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
8,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
9,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.


In [22]:
g.vstack(g)

GenotypeArray((40, 40, 2), dtype=int64)
0/1 ./. 0/1 ./. 0/1  ...  ./. 0/1 ./. 0/1 ./.
0/1 ./. 0/1 ./. 0/1  ...  ./. 0/1 ./. 0/1 ./.
0/1 ./. 0/1 ./. 0/1  ...  ./. 0/1 ./. 0/1 ./.
...
0/1 ./. 0/1 ./. 0/1  ...  ./. 0/1 ./. 0/1 ./.
0/1 ./. 0/1 ./. 0/1  ...  ./. 0/1 ./. 0/1 ./.
0/1 ./. 0/1 ./. 0/1  ...  ./. 0/1 ./. 0/1 ./.

In [23]:
g.hstack(g)

GenotypeArray((20, 80, 2), dtype=int64)
0/1 ./. 0/1 ./. 0/1  ...  ./. 0/1 ./. 0/1 ./.
0/1 ./. 0/1 ./. 0/1  ...  ./. 0/1 ./. 0/1 ./.
0/1 ./. 0/1 ./. 0/1  ...  ./. 0/1 ./. 0/1 ./.
...
0/1 ./. 0/1 ./. 0/1  ...  ./. 0/1 ./. 0/1 ./.
0/1 ./. 0/1 ./. 0/1  ...  ./. 0/1 ./. 0/1 ./.
0/1 ./. 0/1 ./. 0/1  ...  ./. 0/1 ./. 0/1 ./.

In [24]:
g.concatenate(g)

GenotypeArray((40, 40, 2), dtype=int64)
0/1 ./. 0/1 ./. 0/1  ...  ./. 0/1 ./. 0/1 ./.
0/1 ./. 0/1 ./. 0/1  ...  ./. 0/1 ./. 0/1 ./.
0/1 ./. 0/1 ./. 0/1  ...  ./. 0/1 ./. 0/1 ./.
...
0/1 ./. 0/1 ./. 0/1  ...  ./. 0/1 ./. 0/1 ./.
0/1 ./. 0/1 ./. 0/1  ...  ./. 0/1 ./. 0/1 ./.
0/1 ./. 0/1 ./. 0/1  ...  ./. 0/1 ./. 0/1 ./.

In [25]:
g.concatenate(g, axis=1)

GenotypeArray((20, 80, 2), dtype=int64)
0/1 ./. 0/1 ./. 0/1  ...  ./. 0/1 ./. 0/1 ./.
0/1 ./. 0/1 ./. 0/1  ...  ./. 0/1 ./. 0/1 ./.
0/1 ./. 0/1 ./. 0/1  ...  ./. 0/1 ./. 0/1 ./.
...
0/1 ./. 0/1 ./. 0/1  ...  ./. 0/1 ./. 0/1 ./.
0/1 ./. 0/1 ./. 0/1  ...  ./. 0/1 ./. 0/1 ./.
0/1 ./. 0/1 ./. 0/1  ...  ./. 0/1 ./. 0/1 ./.

## HaplotypeArray

In [26]:
h = g.to_haplotypes()

In [27]:
h

HaplotypeArray((20, 80), dtype=int64)
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
...
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .

In [28]:
print(repr(h))

HaplotypeArray((20, 80), dtype=int64)
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
...
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .



In [29]:
print(h)

0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
...
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .



In [30]:
h.display(20, 40)

,0,1,2,3,4,...,75,76,77,78,79
0,0,1,.,.,0,...,.,0,1,.,.
1,0,1,.,.,0,...,.,0,1,.,.
2,0,1,.,.,0,...,.,0,1,.,.
3,0,1,.,.,0,...,.,0,1,.,.
4,0,1,.,.,0,...,.,0,1,.,.
5,0,1,.,.,0,...,.,0,1,.,.
6,0,1,.,.,0,...,.,0,1,.,.
7,0,1,.,.,0,...,.,0,1,.,.
8,0,1,.,.,0,...,.,0,1,.,.
9,0,1,.,.,0,...,.,0,1,.,.


In [31]:
h.displayall()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79
0,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.
1,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.
2,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.
3,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.
4,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.
5,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.
6,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.
7,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.
8,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.
9,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.


In [32]:
h.hstack(h)

HaplotypeArray((20, 160), dtype=int64)
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
...
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .

In [33]:
h.vstack(h)

HaplotypeArray((40, 80), dtype=int64)
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
...
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .

In [34]:
h.concatenate(h)

HaplotypeArray((40, 80), dtype=int64)
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
...
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .

In [35]:
h.concatenate(h, axis=1)

HaplotypeArray((20, 160), dtype=int64)
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
...
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .
0 1 . . 0 1 . . 0 1  ...  . . 0 1 . . 0 1 . .

## AlleleCountsArray

In [36]:
ac = g.count_alleles()
ac

AlleleCountsArray((20, 2), dtype=int32)
20 20
20 20
20 20
...
20 20
20 20
20 20

In [37]:
ac.displayall()

,0,1
0,20,20
1,20,20
2,20,20
3,20,20
4,20,20
5,20,20
6,20,20
7,20,20
8,20,20
9,20,20


In [38]:
print(ac)

20 20
20 20
20 20
...
20 20
20 20
20 20



In [39]:
print(repr(ac))

AlleleCountsArray((20, 2), dtype=int32)
20 20
20 20
20 20
...
20 20
20 20
20 20



In [40]:
ac + 5

AlleleCountsArray((20, 2), dtype=int32)
25 25
25 25
25 25
...
25 25
25 25
25 25

In [41]:
ac + ac

AlleleCountsArray((20, 2), dtype=int32)
40 40
40 40
40 40
...
40 40
40 40
40 40

In [42]:
ac.vstack(ac)

AlleleCountsArray((40, 2), dtype=int32)
20 20
20 20
20 20
...
20 20
20 20
20 20

In [43]:
ac.hstack(ac)

array([[20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20]], dtype=int32)

In [44]:
ac.concatenate(ac)

AlleleCountsArray((40, 2), dtype=int32)
20 20
20 20
20 20
...
20 20
20 20
20 20

In [45]:
ac.concatenate(ac, axis=1)

array([[20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20]], dtype=int32)

## VariantTable

In [46]:
pos = np.unique(np.random.randint(0, 1000000, size=50000))
ref = np.random.choice([b'A', b'C', b'T', b'G'], size=pos.shape[0])
alt = np.random.choice([b'A', b'C', b'T', b'G'], size=pos.shape[0])
ra = np.rec.fromarrays([pos, ref, alt], names=['POS', 'REF', 'ALT'])
vtbl = allel.VariantTable(ra)
vtbl

VariantTable((48784,), dtype=(numpy.record, [('POS', '<i8'), ('REF', 'S1'), ('ALT', 'S1')]))
[(9, b'A', b'A') (25, b'A', b'C') (28, b'C', b'T') ...,
 (999957, b'C', b'C') (999970, b'G', b'T') (999985, b'A', b'C')]

In [47]:
vtbl[:5]

,POS,REF,ALT
0,9,b'A',b'A'
1,25,b'A',b'C'
2,28,b'C',b'T'
3,45,b'T',b'A'
4,49,b'A',b'C'


In [48]:
vtbl[:20].displayall()

,POS,REF,ALT
0,9,b'A',b'A'
1,25,b'A',b'C'
2,28,b'C',b'T'
3,45,b'T',b'A'
4,49,b'A',b'C'
5,84,b'G',b'T'
6,165,b'G',b'T'
7,167,b'G',b'C'
8,197,b'G',b'T'
9,209,b'C',b'A'


In [49]:
vtbl.take(list(range(5)))

,POS,REF,ALT
0,9,b'A',b'A'
1,25,b'A',b'C'
2,28,b'C',b'T'
3,45,b'T',b'A'
4,49,b'A',b'C'


In [50]:
vtbl.compress(np.random.randint(0, 2, size=vtbl.shape[0]).astype(bool))

VariantTable((24468,), dtype=(numpy.record, [('POS', '<i8'), ('REF', 'S1'), ('ALT', 'S1')]))
[(25, b'A', b'C') (28, b'C', b'T') (45, b'T', b'A') ...,
 (999930, b'A', b'A') (999970, b'G', b'T') (999985, b'A', b'C')]

In [51]:
vtbl.concatenate(vtbl)

VariantTable((97568,), dtype=(numpy.record, [('POS', '<i8'), ('REF', 'S1'), ('ALT', 'S1')]))
[(9, b'A', b'A') (25, b'A', b'C') (28, b'C', b'T') ...,
 (999957, b'C', b'C') (999970, b'G', b'T') (999985, b'A', b'C')]

In [52]:
vtbl[['POS', 'ALT']]

VariantTable((48784,), dtype=(numpy.record, [('POS', '<i8'), ('ALT', 'S1')]))
[(9, b'A') (25, b'C') (28, b'T') ..., (999957, b'C') (999970, b'T')
 (999985, b'C')]

## Legacy

In [7]:
gc = allel.GenotypeChunkedArray(g).copy()
gc

,0,1,2,3,4,...,995,996,997,998,999
0,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
1,0/0,0/0,0/0,0/0,0/0,...,0/0,1/1,0/0,0/0,0/0
2,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
3,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,0/0,0/0
4,0/0,0/0,0/0,0/0,0/0,...,0/0,0/0,0/0,1/1,0/0


In [8]:
gc.take([0, 1, 2], axis=1)

,0,1,2
0,0/0,0/0,0/0
1,0/0,0/0,0/0
2,0/0,0/0,0/0
3,0/0,0/0,0/0
4,0/0,0/0,0/0


In [9]:
hc = gc.to_haplotypes()
hc

,0,1,2,3,4,...,1995,1996,1997,1998,1999
0,0,0,0,0,0,...,0,0,0,0,0
1,0,0,0,0,0,...,0,0,0,0,0
2,0,0,0,0,0,...,0,0,0,0,0
3,0,0,0,0,0,...,0,0,0,0,0
4,0,0,0,0,0,...,0,1,1,0,0


In [10]:
hc.take([0, 1, 2], axis=1)

,0,1,2
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0


In [11]:
ac = g.count_alleles()
ac

,0,1
0,1911,89
1,1895,105
2,1896,104
3,1892,108
4,1892,108


In [12]:
acc = gc.count_alleles()
acc

,0,1
0,1911,89
1,1895,105
2,1896,104
3,1892,108
4,1892,108


In [13]:
gc.is_called()

carray((50000, 1000), bool)
  nbytes: 47.68 MB; cbytes: 743.19 KB; ratio: 65.70
  cparams := cparams(clevel=5, shuffle=True, cname='blosclz')
[[ True  True  True ...,  True  True  True]
 [ True  True  True ...,  True  True  True]
 [ True  True  True ...,  True  True  True]
 ..., 
 [ True  True  True ...,  True  True  True]
 [ True  True  True ...,  True  True  True]
 [ True  True  True ...,  True  True  True]]

In [14]:
gc.to_n_alt()

ChunkedArray((50000, 1000), int8, nbytes=47.7M, cbytes=13.0M, cratio=3.7, cname=blosclz, clevel=5, shuffle=True, chunks=(524, 1000), data=bcolz.carray_ext.carray)

In [19]:
vctbl = allel.VariantChunkedTable(vtbl).copy()
vctbl

POS,REF,ALT
9,b'A',b'G'
28,b'T',b'A'
30,b'A',b'A'
43,b'C',b'T'
53,b'G',b'A'
